In [1]:
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(frame, model):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame.flags.writeable = False
    
    results = model.process(frame)
    
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)    
    
    return frame, results

In [4]:
def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [18]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    cap = cv2.VideoCapture(0)
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        frame, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        draw_landmarks(frame, results)
        cv2.imshow('Video', frame)
        
        if cv2.waitKey(10) & 0xFF == ord('d'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [6]:
len(extract_keypoints(results))

NameError: name 'results' is not defined

In [6]:
DATASET_PATH = os.path.join('dataset')

words = np.array(['hello', 'bye', 'thank you', 'sorry', 'yes', 'no', 'please', 'house'])


rem = np.array(['yes', 'no'])
no_sequences = 30
sequence_length = 30

In [15]:
for word in words:
    for sequence in range(no_sequences):
        os.makedirs(os.path.join(DATASET_PATH, word, str(sequence)))

In [16]:
os.listdir(DATASET_PATH)

['no', 'yes', 'bye', 'house', 'hello', 'please', 'thank you', 'sorry']

In [19]:
word = 'yes'
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for sequence in range(no_sequences):
        for frame_num in range(sequence_length):
            ret, frame = cap.read()

            if not ret:
                break

            frame, results = mediapipe_detection(frame, holistic)
            draw_landmarks(frame, results)
    #         print(results)
            if frame_num == 0:
                cv2.putText(frame, 'STARTING COLLECTION', (120,200),  cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(word, sequence), (15,12),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', frame)
                cv2.waitKey(500)
            else:
                cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(word, sequence), (15,12),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', frame)
                
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATASET_PATH, word, str(sequence), str(frame_num))
            np.save(npy_path, keypoints)
        
        if cv2.waitKey(10) & 0xFF == ord('d'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [20]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

2022-02-06 18:13:49.355629: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tushar/Desktop/Coding/UMIT Hackathon/venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-02-06 18:13:49.355708: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [21]:
label_map = {label:num for num, label in enumerate(words)}
label_map

{'hello': 0,
 'bye': 1,
 'thank you': 2,
 'sorry': 3,
 'yes': 4,
 'no': 5,
 'please': 6,
 'house': 7}

In [22]:
sequences, labels = [], []
for word in words:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATASET_PATH, word, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[word])

In [25]:
len(sequences), len(labels)

(240, 240)

In [26]:
np.array(sequences).shape

(240, 30, 1662)

In [27]:
np.array(labels).shape

(240,)

In [29]:
X = np.array(sequences)
Y = to_categorical(labels).astype(int)

X, Y

(array([[[ 0.58359838,  0.50709748, -0.91587383, ...,  0.11270773,
           0.67797202, -0.06782397],
         [ 0.58353472,  0.50705779, -0.90100288, ...,  0.14666079,
           0.59948075, -0.04949772],
         [ 0.58260578,  0.50720084, -0.90593994, ...,  0.17445646,
           0.54813898, -0.03806796],
         ...,
         [ 0.58439982,  0.49574965, -0.8281129 , ...,  0.19339257,
           0.14920644, -0.02322154],
         [ 0.58444268,  0.49563012, -0.83091182, ...,  0.19875701,
           0.14961232, -0.0189153 ],
         [ 0.58421969,  0.49531758, -0.89252937, ...,  0.20185144,
           0.14821731, -0.01546164]],
 
        [[ 0.58420122,  0.49529532, -0.8884443 , ...,  0.20558926,
           0.14797013, -0.02716651],
         [ 0.58424091,  0.49527502, -0.88806027, ...,  0.20797625,
           0.14334324, -0.02188039],
         [ 0.58422935,  0.49434245, -0.93439353, ...,  0.2098276 ,
           0.14213952, -0.01582468],
         ...,
         [ 0.5874536 ,  0.4929838

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

In [54]:
y_test.shape

(24, 8)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

2022-02-06 21:53:52.633277: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tushar/Desktop/Coding/UMIT Hackathon/venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-02-06 21:53:52.633307: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [56]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [57]:
words.shape[0]

8

In [58]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(words.shape[0], activation='softmax'))

In [59]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [60]:
model.fit(X_train, y_train, epochs=300, callbacks=[tb_callback])

Epoch 1/300
7/7 [==============================] - 3s 59ms/step - loss: 2.1549 - categorical_accuracy: 0.1065
Epoch 2/300
7/7 [==============================] - 0s 60ms/step - loss: 2.0922 - categorical_accuracy: 0.0972
Epoch 3/300
7/7 [==============================] - 0s 60ms/step - loss: 2.0461 - categorical_accuracy: 0.1296
Epoch 4/300
7/7 [==============================] - 0s 61ms/step - loss: 1.9631 - categorical_accuracy: 0.2083
Epoch 5/300
7/7 [==============================] - 0s 61ms/step - loss: 1.8935 - categorical_accuracy: 0.2130
Epoch 6/300
7/7 [==============================] - 0s 61ms/step - loss: 1.7434 - categorical_accuracy: 0.2639
Epoch 7/300
7/7 [==============================] - 0s 61ms/step - loss: 1.7548 - categorical_accuracy: 0.2731
Epoch 8/300
7/7 [==============================] - 0s 60ms/step - loss: 1.6038 - categorical_accuracy: 0.3056
Epoch 9/300
7/7 [==============================] - 0s 62ms/step - loss: 1.4777 - categorical_accuracy: 0.3333
Epoch 10/3

7/7 [==============================] - 0s 62ms/step - loss: 0.3147 - categorical_accuracy: 0.8426
Epoch 148/300
7/7 [==============================] - 0s 67ms/step - loss: 0.3645 - categorical_accuracy: 0.8287
Epoch 149/300
7/7 [==============================] - 0s 66ms/step - loss: 0.3822 - categorical_accuracy: 0.8241
Epoch 150/300
7/7 [==============================] - 0s 67ms/step - loss: 0.4150 - categorical_accuracy: 0.7917
Epoch 151/300
7/7 [==============================] - 0s 68ms/step - loss: 0.3638 - categorical_accuracy: 0.8472
Epoch 152/300
7/7 [==============================] - 0s 68ms/step - loss: 0.2614 - categorical_accuracy: 0.8796
Epoch 153/300
7/7 [==============================] - 0s 64ms/step - loss: 0.2583 - categorical_accuracy: 0.8981
Epoch 154/300
7/7 [==============================] - 0s 65ms/step - loss: 0.2380 - categorical_accuracy: 0.9213
Epoch 155/300
7/7 [==============================] - 0s 65ms/step - loss: 0.2363 - categorical_accuracy: 0.8935
Epoch 

7/7 [==============================] - 0s 61ms/step - loss: 0.0902 - categorical_accuracy: 0.9722
Epoch 294/300
7/7 [==============================] - 0s 61ms/step - loss: 0.1286 - categorical_accuracy: 0.9444
Epoch 295/300
7/7 [==============================] - 0s 62ms/step - loss: 0.1780 - categorical_accuracy: 0.9491
Epoch 296/300
7/7 [==============================] - 0s 61ms/step - loss: 0.1153 - categorical_accuracy: 0.9676
Epoch 297/300
7/7 [==============================] - 0s 60ms/step - loss: 0.0812 - categorical_accuracy: 0.9769
Epoch 298/300
7/7 [==============================] - 0s 62ms/step - loss: 0.0792 - categorical_accuracy: 0.9861
Epoch 299/300
7/7 [==============================] - 0s 61ms/step - loss: 0.0814 - categorical_accuracy: 0.9815
Epoch 300/300
7/7 [==============================] - 0s 60ms/step - loss: 0.0870 - categorical_accuracy: 0.9769


In [61]:
model.save('./model.h5', save_format='h5')

In [8]:
# Testing model

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(words.shape[0], activation='softmax'))

2022-02-06 21:53:56.989722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tushar/Desktop/Coding/UMIT Hackathon/venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-02-06 21:53:56.989753: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-06 21:53:56.989776: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tushar): /proc/driver/nvidia/version does not exist
2022-02-06 21:53:56.990022: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.load_weights('./model.h5')

In [10]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [13]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(words[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if words[np.argmax(res)] != sentence[-1]:
                        sentence.append(words[np.argmax(res)])
                else:
                    sentence.append(words[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
#         cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, words[np.argmax(res)], (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_bas

<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'>
house
<class 'mediapipe.python.solution_base.SolutionOutputs'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'media

<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thank you
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.SolutionOutputs'>
no
<class 'mediapipe.python.solution_base.Solut

<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.Solutio